# Fragestellung 1.4: Wie hat sich der Anteil fossiler Heizungen in den letzten 20 Jahren in einer Gemeinde verändert? 

Um diese Fragen zu beantworten, werden Daten aus dem "Gebäude- und Wohnungsregister" (GWR) verwendet. Die Daten werden nach Gemeinde, Bauperiode und Energiequelle gruppiert. Anschliessend werden die einzelnen Objekte gezählt. Anschliessend wird die resultierende Liste als CSV-Datei exportiert.

## Notebook vorbereiten und Daten importieren

In [1]:
# Zu verwendende Bibliotheken importieren
import pandas as pd

In [2]:
# Daten einlesen und erste Zeilen anschauen
df = pd.read_csv(r'C:\Users\AninaNico\Documents\CAS_SpatialDataAnalytics\Zertifikatsarbeit\Uebung\Daten\GWR/gebaeude_batiment_edificio.csv', sep='\t')
df.head()

,EGID,GDEKT,GGDENR,GGDENAME,EGRID,LGBKR,LPARZ,LPARZSX,LTYP,GEBNR,...,GWAERDATH2,GWAERZW1,GENW1,GWAERSCEW1,GWAERDATW1,GWAERZW2,GENW2,GWAERSCEW2,GWAERDATW2,GEXPDAT
0,101173552,GL,1630,Glarus Nord,CH946922708026,1608.0,654,NaN,NaN,1608555,...,NaN,7610.0,7598.0,869.0,2002-06-26,NaN,NaN,NaN,NaN,2022-09-17
1,101173553,GL,1630,Glarus Nord,CH776978052285,1608.0,493,NaN,NaN,1608554,...,NaN,7650.0,7560.0,869.0,2002-06-26,NaN,NaN,NaN,NaN,2022-09-17
2,101173601,GL,1630,Glarus Nord,CH516984227890,1608.0,657,NaN,NaN,1608562,...,NaN,7630.0,7530.0,869.0,2003-06-30,NaN,NaN,NaN,NaN,2022-09-17
3,101173612,GL,1630,Glarus Nord,CH427001226993,1608.0,662,NaN,NaN,1608576,...,NaN,7610.0,7598.0,869.0,2008-07-17,NaN,NaN,NaN,NaN,2022-09-17
4,101174404,GL,1630,Glarus Nord,CH316942227752,1617.0,2006,NaN,NaN,16172231,...,NaN,7610.0,7598.0,859.0,2021-08-26,NaN,NaN,NaN,NaN,2022-09-17


In [9]:
# Alle Attribute und Datentypen der importierten Daten anschauen.
df.dtypes

EGID            int64
GDEKT          object
GGDENR          int64
GGDENAME       object
EGRID          object
LGBKR         float64
LPARZ          object
LPARZSX       float64
LTYP          float64
GEBNR          object
GBEZ           object
GKODE         float64
GKODN         float64
GKSCE         float64
GSTAT           int64
GKAT          float64
GKLAS         float64
GBAUJ         float64
GBAUM         float64
GBAUP         float64
GABBJ         float64
GAREA         float64
GVOL          float64
GVOLNORM      float64
GVOLSCE       float64
GASTW         float64
GANZWHG       float64
GAZZI         float64
GSCHUTZR      float64
GEBF          float64
GWAERZH1      float64
GENH1         float64
GWAERSCEH1    float64
GWAERDATH1     object
GWAERZH2      float64
GENH2         float64
GWAERSCEH2    float64
GWAERDATH2     object
GWAERZW1      float64
GENW1         float64
GWAERSCEW1    float64
GWAERDATW1     object
GWAERZW2      float64
GENW2         float64
GWAERSCEW2    float64
GWAERDATW2

## Daten vorbereiten

In [10]:
# Die gewünschten Attribute sinnvoll umbenennen und in ein neues Dateframe speichern.
buildings = df.rename(columns={"GDEKT":"Kanton",
                               "GGDENAME": "Gemeinde",
                               "GKODE": "lat",
                               "GKODN": "lon",
                               "GKAT":"Gebaeudekategorie",
                               "GBAUJ":"Baujahr",
                               "GBAUP":"Bauperiode",
                               "GANZWHG":"AnzahlWohnungen",
                               "GEBF":"Energiebezugsflaeche",
                               "GWAERZH1":"Waermeerzeuger",
                               "GENH1":"Energiequelle",
                               "GWAERSCEH1":"InfoquelleHeizung"})

In [11]:
# Die nicht verwendeten Attribute löschen
buildings = buildings.drop(['GGDENR','EGRID','LGBKR','LPARZ','LPARZSX','LTYP','GEBNR','GBEZ','GKSCE','GSTAT','GKLAS','GBAUM','GABBJ','GAREA','GVOL','GVOLNORM','GVOLSCE','GASTW','GAZZI','GSCHUTZR','GWAERDATH1','GWAERZH2','GENH2','GWAERSCEH2','GWAERDATH2','GWAERZW1','GENW1','GWAERSCEW1','GWAERDATW1','GWAERZW2','GENW2','GWAERSCEW2','GWAERDATW2','GEXPDAT'], axis=1)

In [12]:
# Nullwerte anschauen
buildings_null = buildings.isnull().sum()
buildings_null

EGID                        0
Kanton                      0
Gemeinde                    0
lat                         6
lon                         6
Gebaeudekategorie           1
Baujahr                 17692
Bauperiode               4072
AnzahlWohnungen         13538
Energiebezugsflaeche    28079
Waermeerzeuger          10961
Energiequelle           10842
InfoquelleHeizung       10962
dtype: int64

In [13]:
# Die korrekten Bezeichnungen den Heizcodes zuweisen und als Liste speichern.
buildings['Energiequelle'] = buildings.Energiequelle.replace({
                                            7500: 'Keine',
                                            7501: 'Luft',
                                            7510: 'Erdwärme',
                                            7511: 'Erdwärme',
                                            7512: 'Erdwärme',
                                            7513: 'Wasser',
                                            7520: 'Gas',
                                            7530: 'Heizöl',
                                            7540: 'Holz',
                                            7541: 'Holz',
                                            7542: 'Holz',
                                            7543: 'Holz',
                                            7550: 'Abwärme',
                                            7560: 'Elektrizität',
                                            7570: 'Sonne',
                                            7580: 'Fernwärme',
                                            7581: 'Ferwärme',
                                            7582: 'Fernwärme',
                                            7598: 'Unbestimmt',
                                            7599: 'Keine'
                                            })


In [18]:
# Bereinigtes DataFrame anschauen
buildings

,EGID,Kanton,Gemeinde,lat,lon,Gebaeudekategorie,Baujahr,Bauperiode,AnzahlWohnungen,Energiebezugsflaeche,Waermeerzeuger,Energiequelle,InfoquelleHeizung
0,101173552,GL,Glarus Nord,2728994.079,1220577.999,1020.0,2001.0,8020.0,1.0,NaN,7410.0,Unbestimmt,869.0
1,101173553,GL,Glarus Nord,2729002.560,1220562.723,1020.0,2001.0,8020.0,1.0,NaN,7410.0,Unbestimmt,869.0
2,101173601,GL,Glarus Nord,2728624.604,1220458.168,1020.0,2002.0,8020.0,1.0,NaN,7430.0,Heizöl,869.0
3,101173612,GL,Glarus Nord,2728375.697,1220483.098,1020.0,2007.0,8021.0,1.0,NaN,7410.0,Unbestimmt,869.0
4,101174404,GL,Glarus Nord,2724463.782,1216958.423,1020.0,2001.0,8020.0,1.0,NaN,7410.0,Unbestimmt,859.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28074,9041161,GL,Glarus,2723090.045,1212358.237,1060.0,NaN,8019.0,NaN,NaN,7410.0,Unbestimmt,859.0
28075,9052034,GL,Glarus,2723126.304,1214790.395,1060.0,1995.0,8018.0,NaN,NaN,7430.0,Heizöl,869.0
28076,9052035,GL,Glarus,2722501.227,1212632.517,1060.0,1975.0,8015.0,NaN,NaN,7430.0,Heizöl,869.0
28077,9080628,GL,Glarus,2723518.619,1212050.656,1060.0,1969.0,8014.0,NaN,NaN,NaN,NaN,NaN


## Anzahl Gebäude pro Gemeinde, Bauperiode und Energiequelle zählen und als DataFrame speichern

In [19]:
# Zählt alle Gebäude pro Gemeinde, Bauperiode und Energiequelle
Gemeindeliste = buildings[['Gemeinde','Bauperiode','Energiequelle']].value_counts().reset_index()

In [20]:
# Die Spaltennamen des DataFrames neu benennen
Gemeindeliste.columns  = ['Gemeinde','Bauperiode','Energiequelle','Anzahl']
Gemeindeliste

,Gemeinde,Bauperiode,Energiequelle,Anzahl
0,Glarus Süd,8011.0,Holz,1449
1,Glarus,8011.0,Heizöl,624
2,Glarus Nord,8011.0,Holz,590
3,Glarus Nord,8011.0,Heizöl,576
4,Glarus Süd,8011.0,Heizöl,557
...,...,...,...,...
294,Glarus Nord,8017.0,Erdwärme,1
295,Glarus Nord,8017.0,Fernwärme,1
296,Glarus,8013.0,Fernwärme,1
297,Glarus Süd,8019.0,Luft,1


## Daten als CSV-Datei exportieren

In [21]:
# Als CSV-File exportieren
Gemeindeliste.to_csv('Gemeindeliste_1-4.csv',sep=',', encoding="utf-8-sig")